In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving training_all_columnns.csv to training_all_columnns.csv


In [ ]:
import pandas as pd
from datetime import datetime
import os

# Helper function to convert a date column to UTC
# Assuming input date format for UFC is a Unix timestamp (in seconds)
def convert_to_utc(date, is_ufc):
    try:
        if is_ufc:
            return datetime.utcfromtimestamp(int(date)).strftime('%Y-%m-%d %H:%M:%S')
        else:
            # Attempt to parse already formatted dates
            return datetime.strptime(date, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d %H:%M:%S')
    except (ValueError, TypeError):
        # Return the original value if conversion fails
        return date

# Function to process files and create the merged output
def merge_csv_files(file1, file2, output_file):
    merged_data = []

    for file in [file1, file2]:
        # Load CSV file
        df = pd.read_csv(file)

        # Determine the label based on the filename
        label = 0 if 'ct_balanced_posts.csv' in os.path.basename(file) else 1

        # Add the label column
        df['label'] = label

        # Convert dates in the "created_utc" or "created_ufc" column to UTC format
        if 'created_ufc' in df.columns:
            df['created_utc'] = df['created_ufc'].apply(lambda x: convert_to_utc(x, True))
            df.drop(columns=['created_ufc'], inplace=True)
        elif 'created_utc' in df.columns:
            df['created_utc'] = df['created_utc'].apply(lambda x: convert_to_utc(x, False))

        # Append the adjusted dataframe to the merged list
        merged_data.append(df)

    # Concatenate all dataframes
    final_df = pd.concat(merged_data, ignore_index=True)

    # Save the merged dataframe to a new CSV file
    final_df.to_csv(output_file, index=False)

# Input and output file paths
file1 = 'ct_balanced_posts.csv'
file2 = 'sz_balanced_posts.csv'
output_file = 'training_all_columnns.csv'

# Call the function
merge_csv_files(file1, file2, output_file)
print(f"Merged file saved as: {output_file}")

Merged file saved as: training_all_columnns.csv


In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
!pip install optuna
!pip install datasets
import optuna
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the BERT tokenizer
model_name = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

# Load dataset
df = pd.read_csv('training_all_columns.csv')  # Replace with the path to your merged CSV file
def combine_columns(row):
    title = row['title'] if pd.notna(row['title']) else ''
    body = row['body'] if pd.notna(row['body']) else ''
    return title + ' ' + body

df['text'] = df.apply(combine_columns, axis=1)
df = df[['label', 'text']]  # Ensure the dataframe only contains the required columns

df.head()

# Split dataset into training and testing
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Tokenization function
def tokenize_data(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

# Convert pandas dataframes to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_data, batched=True)
test_dataset = test_dataset.map(tokenize_data, batched=True)

# Remove unnecessary columns
train_dataset = train_dataset.remove_columns(["text"]).rename_column("label", "labels")
test_dataset = test_dataset.remove_columns(["text"]).rename_column("label", "labels")
train_dataset.set_format("torch")
test_dataset.set_format("torch")

# Define the compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = torch.argmax(torch.tensor(pred.predictions), dim=1).numpy()

    # Standard metrics
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')

    # Confusion matrix for additional metrics
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'true_positives': tp,
        'false_positives': fp,
        'true_negatives': tn,
        'false_negatives': fn
    }

# Define the objective function for Optuna
def objective(trial):
    # Suggest values for the hyperparameters
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
    num_train_epochs = trial.suggest_int('num_train_epochs', 2, 5)

    # Load a fresh model for each trial
    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

    # Set up training arguments
    training_args = TrainingArguments(
        output_dir='./results',
        evaluation_strategy="no",
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=num_train_epochs,
        weight_decay=0.01,
        learning_rate=learning_rate,
        save_strategy="no",  # Disable checkpoint saving
        report_to=[],  # Disable reporting/logging
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Fine-tune the model
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()
    accuracy = eval_results["eval_accuracy"]

    # Return the accuracy for Optuna to optimize
    return accuracy

# Run the Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=15)  # Adjust n_trials as needed

# Get the best hyperparameters
print("Best hyperparameters: ", study.best_params)


Using device: cuda


Map:   0%|          | 0/25022 [00:00<?, ? examples/s]

Map:   0%|          | 0/6256 [00:00<?, ? examples/s]

[I 2025-01-07 03:27:20,578] A new study created in memory with name: no-name-132c4096-8f66-418d-9fed-89400c9679f5
<ipython-input-7-5e3845c3f5bb>:68: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

...

Step,Training Loss
500,0.616800
1000,0.570400
1500,0.533800
2000,0.450000
2500,0.413000
3000,0.346900


[I 2025-01-07 09:28:05,620] Trial 13 finished with value: 0.6970907928388747 and parameters: {'learning_rate': 1.4420399646229783e-05, 'num_train_epochs': 4}. Best is trial 12 with value: 0.7022058823529411.
<ipython-input-7-5e3845c3f5bb>:68: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `ev

Step,Training Loss
500,0.615300
1000,0.570400
1500,0.532700
2000,0.448500


[I 2025-01-07 09:51:40,758] Trial 14 finished with value: 0.7006074168797954 and parameters: {'learning_rate': 1.566655243473692e-05, 'num_train_epochs': 3}. Best is trial 12 with value: 0.7022058823529411.


Best hyperparameters:  {'learning_rate': 1.4145760631193807e-05, 'num_train_epochs': 3}


In [ ]:
from google.colab import runtime
runtime.unassign()